<p align = "center" draggable=”false” ><img src="https://github.com/AI-Maker-Space/LLM-Dev-101/assets/37101144/d1343317-fa2f-41e1-8af1-1dbb18399719" 
     width="200px"
     height="auto"/>
</p>

<h1 align="center" id="heading">OpenAI Agents SDK - AIM</h1>

In this notebook, we'll go over some of the key features of the OpenAI Agents SDK - as explored through a notebook-ified version of their [Research Bot](https://github.com/openai/openai-agents-python/tree/main/examples/research_bot).

In [1]:
### You don't need to run this cell if you're running this notebook locally. 

#!pip install -qU openai-agents

API Key:

In [2]:
import os 
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

Nest Async:

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Agents

As may be expected, the primary thing we'll do in the Agents SDK is construct Agents!

Agents are constructed with a few basic properties:

- A prompt, which OpenAI is using the language "instruction" for, that determines the behaviour or goal of the Agent
- A model, the "brain" of the Agent

They also typically include an additional property: 

- Tool(s) that equip the Agent with things it can use to get stuff done

### Task 1: Create Planner Agent

Let's start by creating our "Planner Agent" - which will come up with the initial set of search terms that should answer a query provided by the user. 



In [4]:
from pydantic import BaseModel
from agents import Agent

PLANNER_PROMPT = (
    "You are a helpful research assistant. Given a query, come up with a set of web searches to perform" 
    "to best answer the query. Output between 5 and 20 terms to query for."
)

Next, we'll define the data models that our Planner Agent will use to structure its output. We'll create:

1. `WebSearchItem` - A model for individual search items, containing the search query and reasoning
2. `WebSearchPlan` - A container model that holds a list of search items

These Pydantic models will help ensure our agent returns structured data that we can easily process.


In [5]:
class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

Now we'll create our Planner Agent using the Agent class from the OpenAI Agents SDK. This agent will use the instructions defined in `PLANNER_PROMPT` and will output structured data in the form of our WebSearchPlan model. We're using the GPT-4o model for this agent to ensure high-quality search term generation.

> NOTE: When we provide an `output_type` - the model will return a [structured response](https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses).


In [6]:
planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_PROMPT,
    model="gpt-4.1",
    output_type=WebSearchPlan,
)

#### ❓Question #1:

Why is it important to provide a structured response template? (As in: Why are structured outputs helpful/preferred in Agentic workflows?)

##### ✅ Answer:

Structured responses:
+ Provide predictable, manchine-readable formatting, like model objects do
+ Act as an API-like contract between the nodes and tools in the agent workflow
+ Facilitate enforcing constraints like required fields and value types
+ Enhance traceability, debugging, and evaluation

### Task 2: Create Search Agent

Now we'll create our Search Agent, which will be responsible for executing web searches based on the terms generated by the Planner Agent. This agent will take each search query, perform a web search using the `WebSearchTool`, and then summarize the results in a concise format.

> NOTE: We are using the `WebSearchTool`, a hosted tool that can be used as part of an `OpenAIResponsesModel` as outlined in the [documentation](https://openai.github.io/openai-agents-python/tools/). This is based on the tools available through OpenAI's new [Responses API](https://openai.com/index/new-tools-for-building-agents/).

The `SEARCH_PROMPT` below instructs the agent to create brief, focused summaries of search results. These summaries are designed to be 2-3 paragraphs, under 300 words, and capture only the essential information without unnecessary details. The goal is to provide the Writer Agent with clear, distilled information that can be efficiently synthesized into the final report.


In [7]:
SEARCH_PROMPT = (
    "You are a research assistant. Given a search term, you search the web for that term and"
    "produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300"
    "words. Capture the main points. Write succinctly, no need to have complete sentences or good"
    "grammar. This will be consumed by someone synthesizing a report, so its vital you capture the"
    "essence and ignore any fluff. Do not include any additional commentary other than the summary"
    "itself."
)

Now we'll create our Search Agent using the Agent class from the OpenAI Agents SDK. This agent will use the instructions defined in `SEARCH_PROMPT` and will utilize the `WebSearchTool` to perform web searches. We're configuring it with `tool_choice="required"` to ensure it always uses the search tool when processing requests.

> NOTE: We can, as demonstrated, indicate how we want our model to use tools. You can read more about that at the bottom of the page [here](https://openai.github.io/openai-agents-python/agents/)

In [8]:
from agents import WebSearchTool
from agents.model_settings import ModelSettings

search_agent = Agent(
    name="Search agent",
    instructions=SEARCH_PROMPT,
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required"),
)

#### ❓ Question #2: 

What other tools are supported in OpenAI's Responses API?

##### ✅ Answer:

In addition to the Web Search tool used in Task 2, the other tools in the Responses API are:
+ File Search: Allows agents to search and extract information from uploaded files (e.g., PDFs, documents).
+ Computer Use: Enables agents to simulate user interactions with websites (filling out forms, clicking buttons, etc.).
+ Function Calling: Lets agents call predefined functions and external APIs.



### Task 3: Create Writer Agent

Finally, we'll create our Writer Agent, which will synthesize all the research findings into a comprehensive report. This agent takes the original query and the research summaries from the Search Agent, then produces a structured report with follow-up questions.

The Writer Agent will:
1. Create an outline for the report structure
2. Generate a detailed markdown report (5-10 pages)
3. Provide follow-up questions for further research

We'll define the prompt for this agent in the next cell. This prompt will instruct the Writer Agent on how to synthesize research findings into a comprehensive report with follow-up questions.

In [9]:
WRITER_PROMPT = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research "
    "assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words.\n"
    "For the follow-up questions, provide exactly 5 unique questions that would help extend "
    "this research. Do not repeat questions."
)

#### 🏗️ Activity #1: 

This prompt is quite generic - modify this prompt to produce a report that is more personalized to either your personal preference, or more appropriate for a specific use case (eg. law domain research)

Now we'll create our Writer Agent using the Agent class from the OpenAI Agents SDK. This agent will synthesize all the research findings into a comprehensive report. We're configuring it with the `ReportData` output type to structure the response with a short summary, markdown report, and follow-up questions.

In [10]:
# I have chosen to create a reporting prommpt which is suitable for creating reports on legacy sporting events

class ReportData(BaseModel):
    short_summary: str
    """A 3–5 sentence paragraph summarizing the event's legacy, significance, and historical highlights."""

    markdown_report: str
    """
    A detailed, markdown-formatted report including the following sections:
    - **Beginnings**: Origins of the tournament and foundational context
    - **Notable Events**: 2–3 major turning points or iconic moments
    - **Recent History**: Key developments or changes in the last 10–20 years
    - **Cultural Significance** *(optional)*: The sport’s broader impact or influence
    """

    follow_up_questions: list[str]
    """
    A list of questions for further research, each followed by one or more reference URLs.
    Format: 'Question? (See: https://example.com, https://example2.com)'
    """

Now we'll define our Writer Agent using the Agent class from the OpenAI Agents SDK. This agent will take the original query and research summaries, then synthesize them into a comprehensive report with follow-up questions. We've defined a custom output type called `ReportData` that structures the response with a short summary, markdown report, and follow-up questions.

In [11]:
writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_PROMPT,
    model="o3-mini",
    output_type=ReportData,
)

#### ❓ Question #3: 

Why are we electing to use a reasoning model for writing our report?

##### ✅ Answer:

We want it to be able to think through the various contexts collected and combine them into a valuable answer. The process can include:
+ Analysis: Interpreting and prioritizing relevant information from multiple search results
+ Reasoning: Generating insights and resolving contradictions or ambiguity in the input
+ Synthesis: Compose a coherent, insightful, and context-aware summary that aligns with the original user query or task



## Task 4: Create Utility Classes 

We'll define utility classes to help with displaying progress and managing the research workflow. The Printer class below will provide real-time updates on the research process.


The Printer class provides real-time progress updates during the research process. It uses Rich's Live display to show dynamic content with spinners for in-progress items and checkmarks for completed tasks. The class maintains a dictionary of items with their completion status and can selectively hide checkmarks for specific items. This creates a clean, interactive console experience that keeps the user informed about the current state of the research workflow.

In [12]:
from typing import Any

from rich.console import Console, Group
from rich.live import Live
from rich.spinner import Spinner

class Printer:
    def __init__(self, console: Console):
        self.live = Live(console=console)
        self.items: dict[str, tuple[str, bool]] = {}
        self.hide_done_ids: set[str] = set()
        self.live.start()

    def end(self) -> None:
        self.live.stop()

    def hide_done_checkmark(self, item_id: str) -> None:
        self.hide_done_ids.add(item_id)

    def update_item(
        self, item_id: str, content: str, is_done: bool = False, hide_checkmark: bool = False
    ) -> None:
        self.items[item_id] = (content, is_done)
        if hide_checkmark:
            self.hide_done_ids.add(item_id)
        self.flush()

    def mark_item_done(self, item_id: str) -> None:
        self.items[item_id] = (self.items[item_id][0], True)
        self.flush()

    def flush(self) -> None:
        renderables: list[Any] = []
        for item_id, (content, is_done) in self.items.items():
            if is_done:
                prefix = "✅ " if item_id not in self.hide_done_ids else ""
                renderables.append(prefix + content)
            else:
                renderables.append(Spinner("dots", text=content))
        self.live.update(Group(*renderables))

Let's create a ResearchManager class that will orchestrate the research process. This class will:
1. Plan searches based on the query
2. Perform those searches to gather information
3. Write a comprehensive report based on the gathered information
4. Display progress using our Printer class


In [13]:
from __future__ import annotations

import asyncio
import time

from agents import Runner, custom_span, gen_trace_id, trace

class ResearchManager:
    def __init__(self):
        self.console = Console()
        self.printer = Printer(self.console)

    async def run(self, query: str) -> None:
        trace_id = gen_trace_id()
        with trace("Research trace", trace_id=trace_id):
            self.printer.update_item(
                "trace_id",
                f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}",
                is_done=True,
                hide_checkmark=True,
            )

            self.printer.update_item(
                "starting",
                "Starting research...",
                is_done=True,
                hide_checkmark=True,
            )
            search_plan = await self._plan_searches(query)
            search_results = await self._perform_searches(search_plan)
            report = await self._write_report(query, search_results)

            final_report = f"Report summary\n\n{report.short_summary}"
            self.printer.update_item("final_report", final_report, is_done=True)

            self.printer.end()

        print("\n\n=====REPORT=====\n\n")
        print(f"Report: {report.markdown_report}")
        print("\n\n=====FOLLOW UP QUESTIONS=====\n\n")
        unique_questions = []
        seen = set()
        
        for question in report.follow_up_questions:
            if question not in seen:
                unique_questions.append(question)
                seen.add(question)
        
        for i, question in enumerate(unique_questions, 1):
            print(f"{i}. {question}")

    async def _plan_searches(self, query: str) -> WebSearchPlan:
        self.printer.update_item("planning", "Planning searches...")
        result = await Runner.run(
            planner_agent,
            f"Query: {query}",
        )
        self.printer.update_item(
            "planning",
            f"Will perform {len(result.final_output.searches)} searches",
            is_done=True,
        )
        return result.final_output_as(WebSearchPlan)

    async def _perform_searches(self, search_plan: WebSearchPlan) -> list[str]:
        with custom_span("Search the web"):
            self.printer.update_item("searching", "Searching...")
            num_completed = 0
            max_concurrent = 5
            results = []
            
            for i in range(0, len(search_plan.searches), max_concurrent):
                batch = search_plan.searches[i:i+max_concurrent]
                tasks = [asyncio.create_task(self._search(item)) for item in batch]
                
                for task in asyncio.as_completed(tasks):
                    try:
                        result = await task
                        if result is not None:
                            results.append(result)
                    except Exception as e:
                        print(f"Search error: {e}")
                        
                    num_completed += 1
                    self.printer.update_item(
                        "searching", f"Searching... {num_completed}/{len(search_plan.searches)} completed"
                    )
            
            self.printer.mark_item_done("searching")
            return results

    async def _search(self, item: WebSearchItem) -> str | None:
        input = f"Search term: {item.query}\nReason for searching: {item.reason}"
        try:
            result = await Runner.run(
                search_agent,
                input,
            )
            return str(result.final_output)
        except Exception as e:
            print(f"Error searching for '{item.query}': {e}")
            return None

    async def _write_report(self, query: str, search_results: list[str]) -> ReportData:
        self.printer.update_item("writing", "Thinking about report...")
        input = f"Original query: {query}\nSummarized search results: {search_results}"
        
        result = Runner.run_streamed(
            writer_agent,
            input,
        )
        
        update_messages = [
            "Thinking about report...",
            "Planning report structure...",
            "Writing outline...",
            "Creating sections...",
            "Cleaning up formatting...",
            "Finalizing report...",
            "Finishing report...",
        ]

        last_update = time.time()
        next_message = 0
        
        async for event in result.stream_events():
            if time.time() - last_update > 5 and next_message < len(update_messages):
                self.printer.update_item("writing", update_messages[next_message])
                next_message += 1
                last_update = time.time()

        self.printer.mark_item_done("writing")
        return result.final_output_as(ReportData)

#### 🏗️ Activity #2:

Convert the above flow into a flowchart style image (software of your choosing, but if you're not sure which to use try [Excallidraw](https://excalidraw.com/)) that outlines how the different Agents interact with each other. 

> HINT: Cursor's CMD+L (CTRL+L on Windows) would be a helpful way to get a basic diagram that you can add more detail to!

![Research Manager Flow](assets/flowchart.png)

## Task 5: Running Our Agent

Now let's run our agent! The main function below will prompt the user for a research topic, then pass that query to our ResearchManager to handle the entire research process. The ResearchManager will: 

1. Break down the query into search items
2. Search for information on each item
3. Write a comprehensive report based on the search results

Let's see it in action!

In [14]:
async def main() -> None:
    query = input("What would you like to research? ")
    await ResearchManager().run(query)

In [15]:
asyncio.run(main())

Output()



=====REPORT=====


Report: # The Open Championship: A Deep Dive into Golf’s Storied Legacy

The Open Championship—often simply referred to as The Open or the British Open—is not just a golf tournament; it is an institution that has shaped the sport of golf through its long-standing traditions, groundbreaking moments, and evolving character. From its humble beginnings in 1860 to becoming a modern spectacle broadcasted worldwide, the tournament represents the essence of golf’s challenge and heritage. This report provides an in-depth overview of The Open Championship, examining its historical evolution, tournament format, iconic moments, qualification procedures, modern enhancements, and the global impact that continues to captivate fans and players alike.

---

## Table of Contents

1. [Introduction](#introduction)
2. [Historical Background](#historical-background)
3. [Tournament Format and Rules](#tournament-format-and-rules)
4. [Iconic Moments and Champions](#iconic-moments-and-champ

---

## Sample Report in Markdown 

---

# The Open Championship: A Deep Dive into Golf’s Storied Legacy

The Open Championship—often simply referred to as The Open or the British Open—is not just a golf tournament; it is an institution that has shaped the sport of golf through its long-standing traditions, groundbreaking moments, and evolving character. From its humble beginnings in 1860 to becoming a modern spectacle broadcasted worldwide, the tournament represents the essence of golf’s challenge and heritage. This report provides an in-depth overview of The Open Championship, examining its historical evolution, tournament format, iconic moments, qualification procedures, modern enhancements, and the global impact that continues to captivate fans and players alike.

---

## Table of Contents

1. [Introduction](#introduction)
2. [Historical Background](#historical-background)
3. [Tournament Format and Rules](#tournament-format-and-rules)
4. [Iconic Moments and Champions](#iconic-moments-and-champions)
5. [Legacy Venues and Links Courses](#legacy-venues-and-links-courses)
6. [Modern Enhancements: Prize Funds, Broadcasting, and Hospitality](#modern-enhancements)
7. [Qualification and Records](#qualification-and-records)
8. [Recent Developments and Future Prospects](#recent-developments-and-future-prospects)
9. [Conclusion](#conclusion)

---

## Introduction

The Open Championship stands unique among golf’s four major tournaments in that it has preserved much of its historic charm while adapting to the evolving demands of modern sports entertainment. The tournament has a heritage that dates back over 160 years, making it not only an event that tests golfing skills under unpredictable coastal weather conditions but also a living museum of golf tradition. The legacy of The Open is built on a rich tapestry of cultures, styles, and memorable moments that have defined its character through the ages.

In this report, we trace the origins and evolution of The Open Championship, highlight the roles of iconic venues and legendary players, and dissect the intricacies of its tournament format—a blend of tradition and modernity that continues to challenge the best players in the world.

---

## Historical Background

### The Birth of a Tradition

The Open Championship was first played on October 17, 1860, at Prestwick Golf Club in Scotland. Originally designed to crown the champion golfer following the demise of famed golfer Allan Robertson, the tournament featured a modest field of eight professionals contending over three rounds on a 12-hole course. It was in 1860 that Willie Park Sr. emerged as the inaugural champion, a milestone that set the stage for centuries of competitive spirit and sporting excellence.

As the tournament grew in prominence, key landmarks emerged, such as the introduction of the Claret Jug in 1872. This trophy, which replaced the original Challenge Belt, has since become emblematic of golfing success and is deeply interwoven with the tournament’s lore. The early decades of The Open witnessed Scottish dominance, led by figures like Young Tom Morris, whose consecutive victories contributed to the tournament’s emerging mythos.

### Expansion and Global Influence

During the late 19th and early 20th centuries, the tournament’s reach began to expand beyond Scotland, embracing venues across the United Kingdom. Iconic courses such as Royal St George’s, Royal Liverpool, and Carnoustie entered the rotation, each offering distinctive challenges reflective of their local landscapes and coastal influences. Notably, international competition began to gain ground with players like Arnaud Massy, the first non-British winner, who broke new ground in 1907.

The historical panorama of The Open has also been shaped by key eras—the Great Triumvirate of Vardon, Taylor, and Braid during the late 1800s and early 1900s; the American breakthroughs of Walter Hagen and Bobby Jones; and the post-war legends such as Tom Watson and Seve Ballesteros, all of which have enriched the tournament’s narrative.

---

## Tournament Format and Rules

### Competition Structure

The format of The Open Championship has largely remained a 72-hole stroke play competition played over four days, from Thursday to Sunday. In typical fashion, the field is composed of 156 top golfers from around the world. During the initial two rounds, players usually tee off in groups of three—strategically scheduled across both morning and afternoon sessions—to navigate the challenging links courses.

After the first 36 holes, a cut is implemented where only the top 70 players (including ties) progress to the final rounds. This mechanism ensures that the tournament remains both competitive and manageable in terms of player numbers.

### Playoff Procedures

In the event of a tie at 72 holes, The Open adopts a unique playoff format. Traditionally, a four-hole aggregate playoff is employed to determine a winner across a predetermined set of holes. Should the tie persist even after this, the competition escalates to a sudden-death playoff on the iconic 18th hole – a nerve-wracking climax that has produced some of the most dramatic moments in the tournament’s history.

### The Influence of Nature

Unlike many tournaments held on manicured courses, The Open is always played on links courses. These courses are characterized by undulating terrain, deep pot bunkers, thick gorse, and significant exposure to weather elements such as wind and rain. The constant battle against the elements adds an unpredictable variable that challenges players to adapt their strategy depending on the day’s conditions—a central theme that has defined many of The Open’s most memorable contests.

---

## Iconic Moments and Champions

Over its long history, The Open Championship has been home to countless unforgettable moments. The tournament has been witness to dramatic victories and astonishing collapses, serving as a stage upon which legends are both made and tested.

### Legendary Duels

One of the most storied chapters in Open history is the 1977 “Duel in the Sun” at Turnberry, where Tom Watson and Jack Nicklaus engaged in a riveting contest. Watson’s victory by a single stroke remains emblematic of the fierce competitive spirit that defines The Open. Equally memorable was the 1999 Open at Carnoustie, where Jean van de Velde’s collapse on the 18th hole turned what seemed like a comfortable lead into an agonizing defeat, culminating in a playoff won by Paul Lawrie.

### Modern Marvels

In recent years, champions such as Xander Schauffele, Brian Harman, and Cameron Smith have carried the torch, each adding a distinctive chapter to the tournament’s ongoing narrative. Schauffele’s triumph at the 2024 Open Championship at Royal Troon—with a remarkable final-round 65—cemented his status as one of golf’s elite performers during a period when American golfers showcased dominance across major tournaments.

---

## Legacy Venues and Links Courses

### The Importance of the Golfing Landscape

The venues for The Open Championship are among the most hallowed in the sport of golf. Each course is steeped in history and presents its own unique set of challenges that have been honed over the course of centuries. The Old Course at St Andrews, often revered as the “Home of Golf,” has hosted The Open more times than any other venue, contributing immeasurably to the tournament’s mystique.

Other key venues include Royal Troon, Royal Liverpool, Royal St George’s, and Carnoustie, each renowned not only for their storied pasts but also for their challenging layouts, particularly the unpredictable coastal winds and rugged terrain that demand precision and adaptability from all competitors.

### The Links Course Experience

The enduring appeal of the links courses lies in their raw, unmanicured beauty. They provide a natural, challenging backdrop that requires players to contend not only with the course design but also with nature itself. The deep bunkers, undulating greens, and ever-changing weather conditions transform each round into a unique battle against the elements – a scenario that epitomizes the spirit of The Open.

---

## Modern Enhancements: Prize Funds, Broadcasting, and Hospitality

### Evolving Prize Funds and Financial Incentives

In recent years, The Open Championship has evolved into a lucrative event with a significant increase in its prize purse. For instance, the 2023 purse was raised to $16.5 million, with the winner earning over $3 million. The subsequent year saw further increases, reflecting the tournament’s rising stature and the financial commitment of sponsors. Such financial incentives not only reward excellence but also serve to attract a global roster of top-tier talent.

### Broadcasting the Legacy

The tournament’s reach extends far beyond the golf course thanks to robust broadcasting deals and digital streaming options. In the United States, networks such as NBC and Golf Channel ensure comprehensive live coverage, including in-depth analysis and behind-the-scenes insight. International audiences can enjoy seamless access via platforms like Peacock, NOW TV, Sky Sports Golf, and TSN, with the official Open app and website serving as digital hubs for live streaming, player statistics, and exclusive content. This global broadcasting framework plays a crucial role in reinforcing the event’s legacy and expanding its fanbase worldwide.

### Hospitality and Ticketing

The modern spectator experience at The Open Championship has also been enhanced with premium hospitality packages and innovative ticketing options. From exclusive fan memberships like ONE CLUB—offering benefits such as free tee times and competitions—to luxurious hospitality experiences available via authorized partners, the tournament has ensured that attending the event is both accessible and memorable. Ticketing processes now include online ballots, resale platforms, and bespoke travel packages, allowing fans to secure a place at one of the most prestigious events in sports.

---

## Qualification and Records

### Pathways to Participation

The Open Championship employs a multifaceted qualification system that rewards excellence while ensuring a diverse field of competitors. Many players earn automatic exemptions through past victories, world rankings, or strong performances in other major tournaments. Meanwhile, the Open Qualifying Series and regional qualifying events provide opportunities for emerging talents and international players to secure a spot in the field.

### Notable Records and Statistical Milestones

The tournament has been the stage for many records that underline its storied history. Harry Vardon’s six victories remain a high watermark in the annals of The Open, while records such as Old Tom Morris’s 13-stroke margin of victory and Branden Grace’s lowest individual round score of 62 highlight the range of extraordinary performances that have graced the event. These records are a testament not only to individual genius but also to the enduring challenge of the links courses, where even the smallest miscalculation can have profound consequences.

---

## Recent Developments and Future Prospects

### Current and Upcoming Tournaments

Looking ahead, the 153rd Open Championship is scheduled for July 2025 at Royal Portrush Golf Club in Northern Ireland. Modern challenges such as logistical planning and enhanced technological integration are shaping the future of the event. Recent ticketing packages, premium hospitality experiences, and innovative digital strategies have further modernized the tournament while maintaining its traditional core.

### Discussions on Future Venues

In contemporary discussions, there has been talk of potential future venues, including the possibility of returning to legendary courses like Turnberry. These discussions, which sometimes involve high-profile stakeholders such as former politicians and industry magnates, underscore the continuing evolution of the tournament as it balances heritage with modern logistical challenges.

### Broader Impact on the Sport of Golf

The Open Championship remains a crucible where golfing legends are forged and strategic innovations are put to the test. Its significant prize funds, extensive global broadcasting network, and the sheer unpredictability of its playing conditions ensure that The Open continues to have a profound impact on the sport of golf. As technology advances and the landscape of professional sports evolves, The Open Championship is poised to leverage its storied history while embracing future innovations to keep its legacy alive.

---

## Conclusion

The Open Championship stands as a beacon of tradition and excellence in the world of golf. From its inception in 1860 to its current status as a global sporting spectacle, the tournament has continually evolved while staying true to the principles and challenges that define the sport. Its history is rich with iconic moments, legendary duels, unforgettable venues, and groundbreaking performances that together create a narrative of resilience, skill, and passion.

The legacy of The Open is not only preserved in the annals of sports history, but it is also actively lived out on every links course where nature and talent converge. As the tournament prepares for future editions, particularly the forthcoming 2025 event at Royal Portrush, enthusiasts and scholars alike can look forward to yet another chapter that will add to this timeless saga. The Open Championship remains a testament to the enduring spirit of golf—a blend of history, tradition, and modern innovation that continues to inspire players and fans around the globe.

---

*End of Report*


=====FOLLOW UP QUESTIONS=====


1. How has the introduction of digital streaming impacted the global viewership of The Open Championship?
2. What are the main challenges in maintaining the historical links courses used in The Open, and how are they addressed?
3. In what ways have sponsorships and prize fund increases influenced the competitive field and overall prestige of The Open?
4. How do modern qualification pathways compare to earlier methods in terms of inclusiveness and diversity of the player field?
5. What role do weather conditions play in shaping tournament outcomes, and how do players adapt their strategies during unpredictable rounds?